# LME walk-through for RQ1

This notebook walks through the analysis for the first research question of my paper (RQ1, MD regions - decoding analysis).

I'm going to try to explain what I did and why, with code included so the steps can be followed. 

In [13]:
library(readr) # required for raw data import

In [14]:
# First, import the csv file with the decoding results (as raw as possible so cleaning etc. can be done here)

col_types_import <- cols(
  Sub = col_double(),
  decodingModel = col_factor(),
  all.roi = col_factor(),
  feature_decoded = col_factor(),
  decAcc = col_double(),
  attCond = col_factor(),
  WasLocAtt = col_factor(),
  WasFeaAtt = col_factor(),
  CalibrationType = col_factor(),
  feedbackGiven = col_factor(),
  Age = col_double()
)

xdata <- read_csv("lme_data_MDrois_collHem.csv", col_names=TRUE, col_types = col_types_import) 


In [15]:
# to have a quick peek/check of the data imported:

head(xdata)

Sub,decodingModel,all.roi,feature_decoded,decAcc,attCond,WasLocAtt,WasFeaAtt,CalibrationType,feedbackGiven,Age
1,1v2,IPS,C,100.00,aLaF,1,1,0,0,30
2,1v2,IPS,C,87.50,aLaF,1,1,0,0,30
3,1v2,IPS,C,87.50,aLaF,1,1,0,0,24
4,1v2,IPS,C,87.50,aLaF,1,1,0,0,24
5,1v2,IPS,C,68.75,aLaF,1,1,0,0,28
6,1v2,IPS,C,100.00,aLaF,1,1,0,0,23


In [16]:
# data cleaning -----------------------------------------------------------
# not necessary to read through unless interested...



# reset the levels of decoding Model
# changing this now so reference is 1v4 (7.1.21)
decLevels <- c("1v4", "1v3", "2v4", "1v2", "2v3", "3v4")

library(gdata) # reorder factor uses gdata
xdata$decodingModel <- gdata::reorder.factor(xdata$decodingModel, new.order = decLevels) # new order specified above

# reorder factors for feature attended and loc attended (so 0 is reference group)
xdata$WasLocAtt <- gdata::reorder.factor(xdata$WasLocAtt, new.order = c("0", "1"))
# xdata$WasLocAtt <- gdata::reorder.factor(xdata$WasLocAtt, new.order = c("1", "0"))

xdata$WasFeaAtt <- gdata::reorder.factor(xdata$WasFeaAtt, new.order = c("0", "1"))


# levels(xdata$all.roi) # consider changing these levels too...
roiLevels <- c("ACC", "AIFO", "IFJ", "aIFS", "pIFS", "IPS", "PM")

xdata$all.roi <- gdata::reorder.factor(xdata$all.roi, new.order = roiLevels) 

# checked all re-ordered values and they match original

# new order specified above


#creating new version where averaged across decModel (as check)
xdata.decModAvg <- aggregate(decAcc ~ 
                                 Sub + all.roi + feature_decoded + 
                                 WasLocAtt + WasFeaAtt + CalibrationType + 
                                 feedbackGiven + Age, 
                               data = xdata, mean)
# creating new variables... -----------------------------------------------
# removing references to colour for this version

#decodingModel:"1v4", "1v3", "2v4", "1v2", "2v3", "3v4"
# create new variable looking at decoding model split into easy/hard conditions

# might be easier if I separate out
Step3 <- c("1v4")
Step2 <- c("1v3", "2v4")
Step1 <- c("1v2", "2v3", "3v4")

# needs to correspond to order they're entered in in the factor levels of relavent variable

Step3.l <- "easy"
Step2.l <- "mid"
Step1.l <- "hard"

# categorised decoding Models
xdata$decModel_cat <- NA
xdata$decModel_cat <- ifelse(xdata$decodingModel %in% Step1, Step1.l, 
                             ifelse (xdata$decodingModel %in% Step2, Step2.l, Step3.l))
xdata$decModel_cat<- as.factor(xdata$decModel_cat)
xdata$decModel_cat <- gdata::reorder.factor(xdata$decModel_cat, 
                                            new.order = c("easy", "mid", "hard")) 
# new order specified above #c("hard", "mid", "easy")
# reordering to be useful^


# Some reminders on benefits of LMMs and why they were used

While repeated measures ANOVAs can model participant (F1) or item level variability (F2), they cannot simultaneously take both sources of variability into account, so observations within a condition must be collapsed across either items or participants – thereby reducing statistical power. Also missing data here is deal with such that if a single observation is missing, entire case is deleted. Also, ANOVAs cannot provide info about magnitude or direction of an effect – don’t provide individual coefficient estimates for each predictor.

LMMs dissect hierarchical and/or longitudinal data by separating the variance due to random sampling from the main effects
That is, on top of the fixed effects normally used in classic linear models, LMMs resolve correlated residuals by introducing random effects that account for differences among random samples (I think randomly sampling within one person). They resolve heterogeneous variance using specific variance functions - thereby improving estimation accuracy and interpretation of fixed effects 

The notes I’m looked at looks at crossed rather than nested random effects --- mixed effects models with nested random effect structures referred to as ‘hierarchical linear models’ 


Additionally, for our purposes, LMMs...
- allow for model selection
- allow for us to test for influence of extraneous factors in a mixed effects design (e.g., caliType and FB)

In [17]:
# load lme4 package
library(lme4)

# REML

RL said that this should be set to FALSE (i.e., uses ML instead) - this is what I've done below/for this analysis.

This is consistent with forum advice (e.g., https://stats.stackexchange.com/questions/48671/what-is-restricted-maximum-likelihood-and-when-should-it-be-used)

It seems that REML requires that the same fixed effects be specified in both models (so more for comparing random effects than fixed effects). Thus, you cannot compare models that differ in fixed effects if they are fitted by REML rather than ML - this is why it's recommended that you use REML=FALSE if you're trying to do model selection.

I am doing model selection with lme, and specifically model selection of the fixed effects, so using REML=FALSE in all of the following models.


Some additional points on this: When you defined REML = FALSE you used the Maximum Likelihood estimated instead of the Restricted Maximum Likelihood one. The REML estimates try to "factor out" the influence of the fixed effects X before moving into finding the optimal random-effect variance structure (see the thread "What is "restricted maximum likelihood" and when should it be used?" for more detailed information on the matter). Computationally this procedure is essentially done by multiplying both parts of the original LME model equation y=Xβ+Zγ+ϵ by a matrix K such that KX=0, i.e. you change both the original y to Ky as well as the Z to KZ. I strongly suspect that this effected the condition number of the design matrix Z and as such help you out of the numerical hard-place you found yourself in the first place (from https://stats.stackexchange.com/questions/242109/model-failed-to-converge-warning-in-lmer).

# Reference group:

The reference group is important in considering results of lme (as all results for summary() are shown in reference to this reference group). In the data cleaning section of my script above, I set my reference group for the pairwise comparisons as 1v4 (as it is set as the first level):
<br>i.e., 

In [18]:
# not run
# decLevels <- c("1v4", "1v3", "2v4", "1v2", "2v3", "3v4")

<br>And for ROI, it's ACC:
<br>i.e., 
<br>

In [19]:
# not run
# roiLevels <- c("ACC", "AIFO", "IFJ", "aIFS", "pIFS", "IPS", "PM")

And for binary variables (e.g., LocAtt), it is 0:
e.g., 

In [20]:
# not run
# xdata$WasLocAtt = gdata::reorder.factor(xdata$WasLocAtt, new.order = c("0", "1"))

For feature, colour is the reference group (was the default)

Note that this makes some of the results from lme (at least the summary results) not directly interpretable as, in most cases in my analysis at least, the reference group is not meaningful (e.g., for ROI, there is no reason why all MD decoding results should be compared to ACC - we want to compare them with each other). For two-level factors, the results from the lme output are clearly interpretable (as are other situations where the reference group makes sense). 

One way to deal with this - if appropriate for your analysis (it is not in mine), is to centre your variables in the first instance - this will allow you to interpret your results in reference to the "mean" of the group. (also z-transform?)

# RQ 1: Do spatial and feature based attention have an interaction/multiplicative effect on decoding accuracy?
includes factors relevant to the first research question

The (fixed effect) factors to include were as follows:

- WasLocAtt (binary: 0,1)
- WasFeaAtt (binary: 0,1)
- feature_decoded (binary: 0,1 [=C,S respectively])
- all.roi (categorical: 7 levels: "ACC", "AIFO", "IFJ", "aIFS", "pIFS", "IPS", "PM")

We did NOT include *decodingModel* here (as not directly relevant to Q1...)

Output variable (DV) = decoding accuracy (continuous but limited: 0:100)

## Inclusion of Calibration Type and Feedback Given factors

We were also interested in seeing whether calibration type (**CalibrationType**: MQ or MPI) and the provision of trial-by-trial feedback (**feedbackGiven**) had an effect on our results

We weren't sure whether to include CalibrationType and feedbackGiven as fixed or random effects. However, there were only 
two levels in both of these factors (0 or 1 - yes/no, respectively). 

It wasn't clear whether we could model factors with so few levels as a random effect (possibly mentioned by Roger Mundry?)
In any case, I did try to include these as random effects, and they could not be fitted:

In [21]:
lmer_RQ1_cali_FB <- lmer(decAcc ~ WasLocAtt * WasFeaAtt * feature_decoded * all.roi + 
                        (1|CalibrationType) + # note - (1|factor) is how to include random intercept
                      (1|feedbackGiven) +
                        (1|Sub),
                      data = xdata, REML = FALSE)

# note singular fit warning


boundary (singular) fit: see ?isSingular


In [22]:
# also tried with different optimiser (see later for more of an explanation on this)
lmer_RQ1_cali_FB <- lmer(decAcc ~ WasLocAtt * WasFeaAtt * feature_decoded * all.roi + 
                        (1|CalibrationType) + # note - (1|factor) is how to include random intercept
                      (1|feedbackGiven) +
                        (1|Sub),
                      data = xdata, REML = FALSE,
                         lmerControl(optimizer ='optimx',
                                  optCtrl=list(method='nlminb')))

# still did not work

# and calibration and FB separately:

lmer_RQ1_cali <- lmer(decAcc ~ WasLocAtt * WasFeaAtt * feature_decoded * all.roi + 
                        (1|CalibrationType) + 
                        (1|Sub),
                      data = xdata, REML = FALSE,
                         lmerControl(optimizer ='optimx',
                                  optCtrl=list(method='nlminb')))

lmer_RQ1_FB <- lmer(decAcc ~ WasLocAtt * WasFeaAtt * feature_decoded * all.roi + 
                        (1|feedbackGiven) +
                        (1|Sub),
                      data = xdata, REML = FALSE,
                         lmerControl(optimizer ='optimx',
                                  optCtrl=list(method='nlminb')))

# none of these were fitted

Warning message in optwrap(optimizer, devfun, getStart(start, rho$pp), lower = rho$lower, :
"convergence code 1 from optimx: none"boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular


In [23]:
# finally, tried with version that I will use later (with random slope included) for completeness:

lmer_RQ1_cali_FB <- lmer(decAcc ~ WasLocAtt * WasFeaAtt * feature_decoded * all.roi + 
                        (1|CalibrationType) + # note - (1|factor) is how to include random intercept
                      (1|feedbackGiven) +
                        (WasLocAtt * WasFeaAtt|Sub),
                      data = xdata, REML = FALSE,
                         lmerControl(optimizer ='optimx',
                                  optCtrl=list(method='nlminb')))

# also save singular fit


boundary (singular) fit: see ?isSingular


Given the above, we included Calibration type and FBgiven as **FIXED** effects

# Random effects and slopes:

Recall that the factors we include as random effects as those factors which we're interested in understanding/knowing how they influence the VARIANCE of a response (while fixed effects are the ones we're interested in with regard to how they influence the MEAN of a response).

Random effects should have at least 5 or 6 levels (which is why the inclusion of calibration type and FB given could not be included as random effects).

My random effect is Subject.

Allowing for individuals to vary (random intercepts) accounts for model to estimate each participant’s deviation from fixed estimate of mean RT, for instance. So in my instance, will allow for differences in decoding accuracy across participants.


# What about random slopes? 
Including roi as a random slope would mean that we might expect the effect of attention condition on decoding to differ according to the roi – so effect of attention on decoding accuracy might be stronger in some areas – would perhaps expect this, so perhaps reasonable (did NOT include this, however).

We did consider including the attention effect across participants as a random slope – as difference across attentional condition might differ between participants -- this was tested below


In [24]:
# To explore whether should just include random intercept for Sub, or whether we should include an attention effect 
# random slope, we ran the following...

# withOUT slope - intercept only (i.e., (1|Sub))
lmer_RQ1_noS <- lmer(decAcc ~ WasLocAtt * WasFeaAtt * feature_decoded * all.roi + 
                        CalibrationType + feedbackGiven +
                        (1|Sub),
                      data = xdata, REML = FALSE,
                      lmerControl(optimizer ='optimx',
                                  optCtrl=list(method='nlminb')))

# WITH random slope (i.e., (WasLocAtt * WasFeaAtt|Sub))
lmer_RQ1_wS <- lmer(decAcc ~ WasLocAtt * WasFeaAtt * feature_decoded * all.roi + 
                        CalibrationType + feedbackGiven +
                        (WasLocAtt * WasFeaAtt|Sub),
                      data = xdata, REML = FALSE,
                      lmerControl(optimizer ='optimx',
                                  optCtrl=list(method='nlminb')))

# to compare:
anova(lmer_RQ1_noS, lmer_RQ1_wS)

# Results suggest that the version WITH slope explains more variance than the one without 
# (and AIC slightly lower with the slope included (wS), despite more parameters included)
# thus, the random slope was included in the full model


,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
lmer_RQ1_noS,60,88614.68,89047.78,-44247.34,88494.68,NA,NA,NA
lmer_RQ1_wS,69,88096.08,88594.14,-43979.04,87958.08,536.6006,9,8.296048e-110


# Optimizers

I have used optimizer ='optimx' in the below - the reasons for this are not well founded. Essentially one of my models did not converge with the default, and suggestions on forums said that different optimisers are worth considering in such cases - with "optimx" being one such alternative. Changing the optimiser worked for one of my models and for consistency, I have used it with all of my models. 

I have tested the difference between using the optimiser and using the default and the difference was minimal (see below). 

In [25]:
lmer_RQ1_def <- lmer(decAcc ~ WasLocAtt * WasFeaAtt * feature_decoded * all.roi + 
                        CalibrationType + feedbackGiven +
                        (WasLocAtt * WasFeaAtt|Sub),
                      data = xdata, REML = FALSE)

lmer_RQ1_opt <- lmer(decAcc ~ WasLocAtt * WasFeaAtt * feature_decoded * all.roi + 
                        CalibrationType + feedbackGiven +
                        (WasLocAtt * WasFeaAtt|Sub),
                      data = xdata, REML = FALSE,
                      lmerControl(optimizer ='optimx',
                                  optCtrl=list(method='nlminb')))

anova(lmer_RQ1_def, lmer_RQ1_opt)
# anova results suggest no difference between the optimisers


,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
lmer_RQ1_def,69,88096.08,88594.14,-43979.04,87958.08,NA,NA,NA
lmer_RQ1_opt,69,88096.08,88594.14,-43979.04,87958.08,2.294517e-06,0,NA


In [26]:
# can also compare output (from summary()) for each model
# -- this shows minimal differences between the optimisers

# here for the default version:
summary(lmer_RQ1_def)



Correlation matrix not shown by default, as p = 58 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it



Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: decAcc ~ WasLocAtt * WasFeaAtt * feature_decoded * all.roi +  
    CalibrationType + feedbackGiven + (WasLocAtt * WasFeaAtt |      Sub)
   Data: xdata

     AIC      BIC   logLik deviance df.resid 
 88096.1  88594.1 -43979.0  87958.1    10011 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.0723 -0.6913  0.0168  0.7211  2.8499 

Random effects:
 Groups   Name                  Variance Std.Dev. Corr             
 Sub      (Intercept)            31.46    5.609                    
          WasLocAtt1             63.47    7.967   -0.77            
          WasFeaAtt1             75.18    8.671   -0.79  0.51      
          WasLocAtt1:WasFeaAtt1 143.87   11.995    0.62 -0.80 -0.61
 Residual                       351.79   18.756                    
Number of obs: 10080, groups:  Sub, 30

Fixed effects:
                                                   Estimate Std. Error t value
(Intercept)                 

In [27]:
# here for the optimiser version:
summary(lmer_RQ1_opt)


Correlation matrix not shown by default, as p = 58 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it



Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: decAcc ~ WasLocAtt * WasFeaAtt * feature_decoded * all.roi +  
    CalibrationType + feedbackGiven + (WasLocAtt * WasFeaAtt |      Sub)
   Data: xdata
Control: lmerControl(optimizer = "optimx", optCtrl = list(method = "nlminb"))

     AIC      BIC   logLik deviance df.resid 
 88096.1  88594.1 -43979.0  87958.1    10011 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.0723 -0.6912  0.0168  0.7211  2.8499 

Random effects:
 Groups   Name                  Variance Std.Dev. Corr             
 Sub      (Intercept)            31.46    5.609                    
          WasLocAtt1             63.44    7.965   -0.77            
          WasFeaAtt1             75.18    8.671   -0.79  0.51      
          WasLocAtt1:WasFeaAtt1 143.81   11.992    0.62 -0.80 -0.61
 Residual                       351.79   18.756                    
Number of obs: 10080, groups:  Sub, 30

Fixed effects:
                             

## Running RQ1 model

#### *Now we're ready to run our full model!*

All core factors included in the full model, with all interactions allowed for factors of interest 
(i.e., CalibrationType and feedbackGiven only included as possible main fixed effects)
- Random effects slope of attention effect over subjects included
- optimiser specified and ML fitted


In [28]:
lmer_RQ1_full <- lmer(decAcc ~ WasLocAtt * WasFeaAtt * feature_decoded * all.roi + 
                        CalibrationType + feedbackGiven +
                        (WasLocAtt * WasFeaAtt|Sub),
                      data = xdata, REML = FALSE,
                      lmerControl(optimizer ='optimx',
                                  optCtrl=list(method='nlminb')))

# you can look at the main results of this with summary()
summary(lmer_RQ1_full)


Correlation matrix not shown by default, as p = 58 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it



Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: decAcc ~ WasLocAtt * WasFeaAtt * feature_decoded * all.roi +  
    CalibrationType + feedbackGiven + (WasLocAtt * WasFeaAtt |      Sub)
   Data: xdata
Control: lmerControl(optimizer = "optimx", optCtrl = list(method = "nlminb"))

     AIC      BIC   logLik deviance df.resid 
 88096.1  88594.1 -43979.0  87958.1    10011 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.0723 -0.6912  0.0168  0.7211  2.8499 

Random effects:
 Groups   Name                  Variance Std.Dev. Corr             
 Sub      (Intercept)            31.46    5.609                    
          WasLocAtt1             63.44    7.965   -0.77            
          WasFeaAtt1             75.18    8.671   -0.79  0.51      
          WasLocAtt1:WasFeaAtt1 143.81   11.992    0.62 -0.80 -0.61
 Residual                       351.79   18.756                    
Number of obs: 10080, groups:  Sub, 30

Fixed effects:
                             

# Model selection

There are a few ways to look at which models to include. A simple way (and the main approach I've used) is to use the step function (included in the lmerTest package) with your full model. This runs a backward elimination of random-effect terms followed by backward elimination of fixed-effect terms in the specified linear mixed model. 

Ultimately, you get a table indicating which random and fixed effects to eliminate and which to keep - essentially, the factors to include in your final model. 


In [29]:
# test with step
library(lmerTest) # need for step fnc

Warning message:
"package 'lmerTest' was built under R version 3.6.3"
Attaching package: 'lmerTest'

The following object is masked from 'package:lme4':

    lmer

The following object is masked from 'package:stats':

    step



In [30]:
step_res <- step(lmer_RQ1_full) # seems to have some problems here, but not on my local PC

# display elimination results
step_res 

ERROR: Error in `$<-`(`*tmp*`, formula, value = Terms): no method for assigning subsets of this S4 class


--

The above results suggest the elimination of six factors: 

> - *feedbackGiven*                                      
> - *WasLocAtt:WasFeaAtt:feature_decoded:all.roi*         
> - *WasLocAtt:feature_decoded:all.roi*                  
> - *WasFeaAtt:feature_decoded:all.roi*                  
> - *feature_decoded:all.roi*                             
> - *CalibrationType*                                     

Leaving this as the final model (assigned later to **lmer_RQ1_final**):

> <font color='green'>
decAcc ~ WasLocAtt + WasFeaAtt + feature_decoded + all.roi + 
<br> (WasLocAtt * WasFeaAtt | Sub) + 
<br> WasLocAtt:WasFeaAtt + WasLocAtt:feature_decoded + 
<br> WasFeaAtt:feature_decoded + 
<br> WasLocAtt:all.roi + 
<br> WasFeaAtt:all.roi + 
<br> WasLocAtt:WasFeaAtt:feature_decoded + 
<br> WasLocAtt:WasFeaAtt:all.roi
</font>


--- 


Another approach suggested to me is a more systematic approach to testing for the inclusion of each factor. This looks at the sequential addition of each factor above that previously specified (so it may not be the best approach as cannot have a higher-order factor included without including all that came before it) --- *not so useful, but including for now for completeness*


In [31]:
# start with no predictors but intercept!!
rq1_baseline <- lmer(decAcc ~ 1 + (WasLocAtt * WasFeaAtt | Sub), data = xdata, REML=FALSE,
                     lmerControl(optimizer ='optimx',
                                 optCtrl=list(method='nlminb')))

#main effects
rq1_WasLocAtt_M <- update(rq1_baseline, .~. + WasLocAtt)  
rq1_WasFeaAtt_M <- update(rq1_WasLocAtt_M, .~. + WasFeaAtt)   
rq1_feature_decoded_M <- update(rq1_WasFeaAtt_M, .~. + feature_decoded) 
rq1_all.roi_M <- update(rq1_feature_decoded_M, .~. + all.roi) # should give same res as likelihood test

#second-order effects
rq1_WasLocAtt_WasFeaAtt <- update(rq1_all.roi_M, .~. + WasLocAtt:WasFeaAtt) 
rq1_WasLocAtt_feature_decoded <- update(rq1_WasLocAtt_WasFeaAtt, .~. + WasLocAtt:feature_decoded) 
rq1_WasFeaAtt_feature_decoded <- update(rq1_WasLocAtt_feature_decoded, .~. + WasFeaAtt:feature_decoded) 
rq1_WasLocAtt_all.roi <- update(rq1_WasFeaAtt_feature_decoded, .~. + WasLocAtt:all.roi) 
rq1_WasFeaAtt_all.roi <- update(rq1_WasLocAtt_all.roi, .~. + WasFeaAtt:all.roi) 
rq1_feature_decoded_all.roi <- update(rq1_WasFeaAtt_all.roi, .~. + feature_decoded:all.roi) 

#third-order effects
rq1_WasLocAtt_WasFeaAtt_feature_decoded <- update(rq1_feature_decoded_all.roi, .~. + 
                                                    WasLocAtt:WasFeaAtt:feature_decoded) 
rq1_WasLocAtt_WasFeaAtt_all.roi <- update(rq1_WasLocAtt_WasFeaAtt_feature_decoded, .~. + 
                                            WasLocAtt:WasFeaAtt:all.roi) 
rq1_WasLocAtt_feature_decoded_all.roi <- update(rq1_WasLocAtt_WasFeaAtt_all.roi, .~. + 
                                            WasLocAtt:feature_decoded:all.roi) 
rq1_WasFeaAtt_feature_decoded_all.roi <- update(rq1_WasLocAtt_feature_decoded_all.roi, .~. + 
                                            WasFeaAtt:feature_decoded:all.roi) 

#fourth-order effects
rq1_WasLocAtt_WasFeaAtt_feature_decoded_all.roi <- update(rq1_WasFeaAtt_feature_decoded_all.roi, .~. + 
                                            WasLocAtt:WasFeaAtt:feature_decoded:all.roi) 



rq1_caliType<- update(rq1_WasLocAtt_WasFeaAtt_all.roi, .~. + CalibrationType) 
rq1_fbGiven<- update(rq1_caliType, .~. + feedbackGiven) 


#now test models...
anova(rq1_baseline, rq1_WasLocAtt_M, rq1_WasFeaAtt_M, rq1_feature_decoded_M, rq1_all.roi_M,
      rq1_WasLocAtt_WasFeaAtt, rq1_WasLocAtt_feature_decoded, rq1_WasFeaAtt_feature_decoded,
      rq1_WasLocAtt_all.roi, rq1_WasFeaAtt_all.roi, rq1_feature_decoded_all.roi,
      rq1_WasLocAtt_WasFeaAtt_feature_decoded, rq1_WasLocAtt_WasFeaAtt_all.roi, 
      rq1_WasLocAtt_feature_decoded_all.roi, rq1_WasFeaAtt_feature_decoded_all.roi,
      rq1_WasLocAtt_WasFeaAtt_feature_decoded_all.roi,
      rq1_caliType, rq1_fbGiven)

# note - does give message here that some issues with convergence with above

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.0023364 (tol = 0.002, component 1)"Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00205704 (tol = 0.002, component 1)"

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
rq1_baseline,12,88330.36,88416.98,-44153.18,88306.36,NA,NA,NA
rq1_WasLocAtt_M,13,88316.96,88410.79,-44145.48,88290.96,1.539989e+01,1,8.699320e-05
rq1_WasFeaAtt_M,14,88306.71,88407.76,-44139.35,88278.71,1.224844e+01,1,4.656473e-04
rq1_feature_decoded_M,15,88292.44,88400.72,-44131.22,88262.44,1.626404e+01,1,5.509968e-05
rq1_all.roi_M,21,88286.10,88437.68,-44122.05,88244.10,1.834733e+01,6,5.420364e-03
rq1_WasLocAtt_WasFeaAtt,22,88252.00,88410.81,-44104.00,88208.00,3.609159e+01,1,1.882579e-09
rq1_WasLocAtt_feature_decoded,23,88217.72,88383.74,-44085.86,88171.72,3.628256e+01,1,1.706847e-09
rq1_WasFeaAtt_feature_decoded,24,88093.24,88266.48,-44022.62,88045.24,1.264810e+02,1,2.412949e-29
rq1_WasLocAtt_all.roi,30,88075.84,88292.39,-44007.92,88015.84,2.940178e+01,6,5.105773e-05
rq1_WasFeaAtt_all.roi,36,88068.90,88328.76,-43998.45,87996.90,1.893451e+01,6,4.275682e-03


--

Based on this, could possibly exclude:
- feature_decoded_all.roi 
- WasLocAtt_feature_decoded_all.roi
- WasFeaAtt_feature_decoded_all.roi
- WasLocAtt_WasFeaAtt_feature_decoded_all.roi
- caliType
- fbGiven

One can see from the above that again, the addition of calibration type and feedback given do not seem to add much to the models.

(Note - these are the SAME factors indicated for possible exclusion with the step function)

In [32]:
# From the above, our final model is...

lmer_RQ1_final <- lmer(decAcc ~ WasLocAtt + WasFeaAtt + feature_decoded + all.roi + 
                       (WasLocAtt * WasFeaAtt | Sub) + 
                       WasLocAtt:WasFeaAtt + WasLocAtt:feature_decoded + WasFeaAtt:feature_decoded + 
                       WasLocAtt:all.roi + WasFeaAtt:all.roi + 
                       WasLocAtt:WasFeaAtt:feature_decoded + WasLocAtt:WasFeaAtt:all.roi,
                       data = xdata, REML = FALSE,
                         lmerControl(optimizer ='optimx',
                                     optCtrl=list(method='nlminb')))


In [33]:
# Can look at the main results of the model here:

summary(lmer_RQ1_final)


Correlation matrix not shown by default, as p = 32 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it



Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: decAcc ~ WasLocAtt + WasFeaAtt + feature_decoded + all.roi +  
    (WasLocAtt * WasFeaAtt | Sub) + WasLocAtt:WasFeaAtt + WasLocAtt:feature_decoded +  
    WasFeaAtt:feature_decoded + WasLocAtt:all.roi + WasFeaAtt:all.roi +  
    WasLocAtt:WasFeaAtt:feature_decoded + WasLocAtt:WasFeaAtt:all.roi
   Data: xdata
Control: lmerControl(optimizer = "optimx", optCtrl = list(method = "nlminb"))

     AIC      BIC   logLik deviance df.resid 
 88057.0  88367.4 -43985.5  87971.0    10037 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.0624 -0.6907  0.0159  0.7180  2.8661 

Random effects:
 Groups   Name                  Variance Std.Dev. Corr             
 Sub      (Intercept)            31.39    5.603                    
          WasLocAtt1             63.44    7.965   -0.77            
          WasFeaAtt1             75.18    8.670   -0.78  0.51      
          WasLocA

# Testing for main effects and interactions

I was recommended a few different ways to do this. The first is Romy's way:

#### Romy's method of testing main effects... F-test

This uses the contest function (from lmerTest). You specify which fixed factor effects you wish to look at here (see fixef() and its use with ROI below):

In [34]:
# An F-test of the main effect of ROI
a <- length(fixef(lmer_RQ1_final))

contest(lmer_RQ1_final, L=diag(a)[5:10, ]) # note - numbers refer to order in fixef of model
#n.s.

Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
1386.657,231.1095,6,9960,0.6561664,0.6852004


I had read, however, that this may not be the best way to test for lower order effects (need to find reference to recall exactly why...). It does seem to give an incorrect result (i.e., other tests suggest there IS a main effect for ROI). I think this is because of how SS is handled when running an F-test in this way. This method does give the same result though for the higher-order interactions.

---

Another method mentioned to me (by Yudong Chen) is the likelihood method. Shown below for the three way interaction between WasLocAtt:WasFeaAtt:all.roi


In [35]:
# For the likelihood method approach, you take your model and drop the terms you wish to exclude to see its effect on the result:

# here, dropping WasLocAtt:WasFeaAtt:all.roi

RQ1_drop1 <- lmer(decAcc ~ WasLocAtt + WasFeaAtt + feature_decoded + all.roi + 
                       (WasLocAtt * WasFeaAtt | Sub) + 
                       WasLocAtt:WasFeaAtt + WasLocAtt:feature_decoded + WasFeaAtt:feature_decoded + 
                       WasLocAtt:all.roi + WasFeaAtt:all.roi + 
                       WasLocAtt:WasFeaAtt:feature_decoded,
                       data = xdata, REML = FALSE,
                         lmerControl(optimizer ='optimx',
                                     optCtrl=list(method='nlminb')))

anova(RQ1_drop1, lmer_RQ1_final, test="Chisq") 

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
RQ1_drop1,37,88066.09,88333.17,-43996.05,87992.09,NA,NA,NA
lmer_RQ1_final,43,88057.01,88367.39,-43985.50,87971.01,21.0874,6,0.00176944


In [55]:
# this is significant!

# Can also compare this to the F-test method:
contest(lmer_RQ1_final, L=diag(a)[27:32, ]) 
# gives a very similar result


Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
7435.097,1239.183,6,9960,3.51829,0.001766342


In [36]:
#Ultimately, though, an ANOVA on the model seems to give the clearest results of the main effects and interactions 
# (note - the results here are the same as the F-test for the higher-order factors)

anova(lmer_RQ1_final)

# I think this is the approach I mainly used

,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
WasLocAtt,32840.313,32840.313,1,30.00040,93.240273,1.026921e-10
WasFeaAtt,21529.164,21529.164,1,29.99423,61.125640,1.006516e-08
feature_decoded,5881.944,5881.944,1,9959.99984,16.700027,4.412379e-05
all.roi,6623.853,1103.975,6,9959.99984,3.134409,4.528817e-03
WasLocAtt:WasFeaAtt,24628.030,24628.030,1,30.01262,69.923944,2.465840e-09
WasLocAtt:feature_decoded,13063.058,13063.058,1,9959.99985,37.088657,1.170488e-09
WasFeaAtt:feature_decoded,45167.550,45167.550,1,9959.99983,128.239785,1.507319e-29
WasLocAtt:all.roi,10417.736,1736.289,6,9959.99984,4.929676,4.806379e-05
WasFeaAtt:all.roi,6692.677,1115.446,6,9959.99983,3.166977,4.184679e-03
WasLocAtt:WasFeaAtt:feature_decoded,1698.304,1698.304,1,9959.99982,4.821828,2.812450e-02


---

*I need to finalise this --- add notes on differences between likelihood method and ANOVAs w.r.t. SS...*


# Post-hoc tests

These seem to function in a similar way to post-hoc tests following an ANOVA. I have used the emmeans package, following RL's recommendation (see https://mran.microsoft.com/snapshot/2018-02-12/web/packages/emmeans/vignettes/interactions.html)

In [37]:
library(emmeans)

In [38]:
# for instance, to look at one of the three-way interactions (SpatialAtt*FeaAtt*feature_decoded):

emms_test <- emmeans(lmer_RQ1_final, ~ WasLocAtt*WasFeaAtt | feature_decoded)

# this gives you the test of the interaction for C and S separately
contrast(emms_test, interaction = "pairwise")


Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 10080' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 10080)' or larger];
but be warned that this may result in large computation time and memory use.
Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 10080' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 10080)' or larger];
but be warned that this may result in large computation time and memory use.


feature_decoded = C:
 WasLocAtt_pairwise WasFeaAtt_pairwise estimate   SE  df z.ratio p.value
 0 - 1              0 - 1                  21.0 2.43 Inf   8.632  <.0001

feature_decoded = S:
 WasLocAtt_pairwise WasFeaAtt_pairwise estimate   SE  df z.ratio p.value
 0 - 1              0 - 1                  17.7 2.43 Inf   7.281  <.0001

Results are averaged over the levels of: all.roi 
Degrees-of-freedom method: asymptotic 

In [39]:
# and for the other three-way interaction (i.e., SpatialAtt*FeaAtt*all.roi):

emms_test <- emmeans(lmer_RQ1_final, ~ WasLocAtt*WasFeaAtt | all.roi)

# this gives you the test of the interaction for C and S separately
contrast(emms_test, interaction = "pairwise")

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 10080' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 10080)' or larger];
but be warned that this may result in large computation time and memory use.
Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 10080' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 10080)' or larger];
but be warned that this may result in large computation time and memory use.


all.roi = ACC:
 WasLocAtt_pairwise WasFeaAtt_pairwise estimate   SE  df z.ratio p.value
 0 - 1              0 - 1                  20.6 2.95 Inf   6.978  <.0001

all.roi = AIFO:
 WasLocAtt_pairwise WasFeaAtt_pairwise estimate   SE  df z.ratio p.value
 0 - 1              0 - 1                  21.4 2.95 Inf   7.243  <.0001

all.roi = IFJ:
 WasLocAtt_pairwise WasFeaAtt_pairwise estimate   SE  df z.ratio p.value
 0 - 1              0 - 1                  24.6 2.95 Inf   8.337  <.0001

all.roi = aIFS:
 WasLocAtt_pairwise WasFeaAtt_pairwise estimate   SE  df z.ratio p.value
 0 - 1              0 - 1                  17.1 2.95 Inf   5.801  <.0001

all.roi = pIFS:
 WasLocAtt_pairwise WasFeaAtt_pairwise estimate   SE  df z.ratio p.value
 0 - 1              0 - 1                  19.4 2.95 Inf   6.572  <.0001

all.roi = IPS:
 WasLocAtt_pairwise WasFeaAtt_pairwise estimate   SE  df z.ratio p.value
 0 - 1              0 - 1                  19.7 2.95 Inf   6.660  <.0001

all.roi = PM:
 WasLocAtt_